In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from preprocessing_utils import process_sat_data, pickle_save, pickle_load

In [3]:
data_train = pd.read_csv('../data/train.csv')
data_test = pd.read_csv('../data/track1/test.csv')

In [4]:
train_data = data_train.copy(deep=True)
test_data = data_test.copy(deep=True)

In [5]:
process_sat_data(train_data, test_data, scale=10000)